## Preprocessing

### Data preparation
- load raw post data from json file
- convert json data into dataframe
- merge post 'text' with emotion labels
- split data into train/test

In [ ]:
import pandas as pd
import json

path = "/kaggle/input/dm-lab-2-private-competition"

# load raw post data from json
with open(f"{path}/final_posts.json", "r", encoding = "utf-8") as f:
    raw_posts = json.load(f)

# json to dataframe
posts = []
for item in raw_posts:
    post = item["root"]["_source"]["post"]
    posts.append({
        "id": post["post_id"],
        "text": post["text"],
        "hashtags": " ".join(post.get("hashtags", []))
    })
posts_df = pd.DataFrame(posts)

# load emotion labels and data split
emotion_df = pd.read_csv(f"{path}/emotion.csv")
split_df = pd.read_csv(f"{path}/data_identification.csv")

# train/test dataframe
merged = posts_df.merge(split_df, on = "id", how = "inner")
train_df = merged[merged["split"] == "train"].merge(emotion_df, on = "id", how = "left")
test_df = merged[merged["split"] == "test"]

print("train_df:", train_df.shape)
print("test_df:", test_df.shape)

print(train_df.head())
print(test_df.head())

train_df: (47890, 5)
test_df: (16281, 4)
         id                                               text hashtags  \
0  0x35663e  I bet there is an army of married couples who ...            
1  0xc78afe                         This could only end badly.            
2  0x90089c  My sister squeezed a lime in her milk when she...            
3  0x2ffb63                                Thank you so much❤️            
4  0x989146  Stinks because ive been in this program for a ...            

   split emotion  
0  train     joy  
1  train    fear  
2  train     joy  
3  train     joy  
4  train     joy  
         id                                               text hashtags split
0  0x61fc95  We got the ranch, loaded our guns and sat up t...           test
4  0xaba820         and that got my head bobbing a little bit.           test
5  0x66e44d                Same. Glad it's not just out store.           test
6  0xc03cf5  Like always i will wait and see thanks for the...           test
8  0

### Basic checks

Basic check for missing values

In [2]:
print("train_df missing values:")
print(train_df.isna().sum())

print("\ntest_df missing values:")
print(test_df.isna().sum())

train_df missing values:
id          0
text        0
hashtags    0
split       0
emotion     0
dtype: int64

test_df missing values:
id          0
text        0
hashtags    0
split       0
dtype: int64


Observe the class distribution of the emotion labels in the training set.

In [3]:
print(train_df["emotion"].value_counts())

emotion
joy         23797
anger       10694
surprise     6281
sadness      3926
fear         2009
disgust      1183
Name: count, dtype: int64


### Text preprocessing
Analyze the common patterns, such as emails, user mentions, and multiple whitespace.

In [4]:
import re

# count occurrences of pattern
def count_pattern(df, pattern):
    return df["text"].str.contains(pattern, regex = True).sum()

print("\nnumber of rows with emails")
print(count_pattern(train_df, r"\S+@\S+"))

print("\nnumber of rows with @usernames")
print(count_pattern(train_df, r"@\w+"))

print("\nnumber of rows with multiple whitespaces")
print(count_pattern(train_df, r"\s{2,}"))


number of rows with emails
60

number of rows with @usernames
2410

number of rows with multiple whitespaces
596


Use the **clean_text** function to remove emails, user mentions, and compress multiple whitespaces.

In [5]:
def clean_text(s):
    #s = s.lower()
    s = re.sub(r"\S+@\S+", "", s) # remove emails
    s = re.sub(r"@\w+", "", s) # remove @usernames
    s = re.sub(r"\s+", " ", s).strip() # compress multiple spaces into one
    return s

# apply the function and store results in a new column 'clean_text'
train_df.loc[:, "clean_text"] = train_df["text"].apply(clean_text)
test_df.loc[:, "clean_text"] = test_df["text"].apply(clean_text)

print(train_df[["text", "clean_text", "emotion"]].sample(10, random_state = 42))
print("\ntrain_df:", train_df.shape)
print("test_df:", test_df.shape)

                                                    text  \
32473               >YOUR STATEMENT I'm not that person.   
2494    Do you have any relatives who could support you?   
41963                                       Wtf? Really?   
29666  In our county, 2018 was the turning point—then...   
15806  All good brother. Los Lonely Boys just came on...   
42109      Haven't printed a retraction in twenty years!   
47596  Heroin definetley isn't safer than Ritalin. Th...   
37756  I like your colour choices and I can feel your...   
5697   I'd enjoy a nice warm hug. Along with a deligh...   
3564                                  A royal with creme   

                                              clean_text   emotion  
32473               >YOUR STATEMENT I'm not that person.       joy  
2494    Do you have any relatives who could support you?       joy  
41963                                       Wtf? Really?     anger  
29666  In our county, 2018 was the turning point—then...     an

/tmp/ipykernel_47/2677140072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, "clean_text"] = test_df["text"].apply(clean_text)


a sample of original and cleaned text pairs

In [6]:
def show_cleaning_examples(df, pattern, description):
    print(f"\n{description}")
    mask = df["text"].str.contains(pattern, regex = True, na = False)
    samples = df[mask].head(5)
    
    for idx, (_, row) in enumerate(samples.iterrows(), 1):
        print(f"\nsample {idx}")
        print("original:", row["text"])
        print("cleaned :", row["clean_text"])
    
    print("-" * 20)

show_cleaning_examples(train_df, r"\S+@\S+", "remove email")
show_cleaning_examples(train_df, r"@\w+", "remove @user")
show_cleaning_examples(train_df, r"\s{2,}", "compress multiple spaces")



remove email

sample 1
original: Profit are from 5-20% per day to 100-300% per month.Don't hesitate to contact us for the more results! info@fxventury.com
cleaned : Profit are from 5-20% per day to 100-300% per month.Don't hesitate to contact us for the more results!

sample 2
original: Go follow #beautiful #Snowgang ♥@Amynicolehill12 ♥ #Princess #fitness #bodyposi #haircut #smile #Whitegirlwednesday
cleaned : Go follow #beautiful #Snowgang ♥ #Princess #fitness #bodyposi #haircut #smile #Whitegirlwednesday

sample 3
original: .@billradkeradio is not a fan of The Beat Happening. But that's not to discourage aspiring other Olympia musicians! #KUOWrecord
cleaned : is not a fan of The Beat Happening. But that's not to discourage aspiring other Olympia musicians! #KUOWrecord

sample 4
original: .@Travelanswerman: The possibilities R endless Becca when U unplug from the 'Matrix!' Stay salty and #happy #surfing! #sunset #aloha …
cleaned : The possibilities R endless Becca when U unplug from 

## Feature Engineering

### label mapping
Creating a mapping between emotion strings and integer labels (label2id and id2label).
- label2id: Maps emotion to integer, the result will be:
    - **label2id: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}**
- id2label: Maps integer back to emotion.

In [7]:
import numpy as np

df = train_df.copy()

# label-to-id and id-to-label mapping
unique_labels = sorted(df["emotion"].unique())
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print("label2id:", label2id)
df["label"] = df["emotion"].map(label2id)

train_df_bert = df
print("Full train_df_bert shape:", train_df_bert.shape)

label2id: {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'sadness': 4, 'surprise': 5}
Full train_df_bert shape: (47890, 7)


## Model Implementation

### Tokenizer and dataset class

- Load the **tokenizer** for the pre-trained model roberta-base. It can convert text to tokens that model can understand.
    - In NLP course in NTHU, one assignment has experienced that **RoBERTa has better performance than BERT**, so I choose this.
- Define **EmotionDataset class** to handle text and label.

In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset

# define the model and load the tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# PyTorch dataset class for emotion classification
class EmotionDataset(Dataset):
    def __init__(self, df):
        self.texts = df["clean_text"].tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return{
            "text": self.texts[idx],
            "label": int(self.labels[idx])
        }

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### Define a collate_fn

This function will be used by DataLoader to process samples(batch) from EmotionDataset.

In [ ]:
def collate_fn(batch):
    # extract text
    texts = [item["text"] for item in batch]

    # tokenize the batch of texts
    enc = tokenizer(
        texts,
        padding = "longest",
        truncation = True,
        max_length = 256,
        return_tensors = "pt"
    )

    # for train/valid
    if "label" in batch[0]:
        enc["labels"] = torch.tensor([item["label"] for item in batch], dtype = torch.long)
    return enc

### k-fold cross-validation
to ensure a robust training

- split: divide data into train/validation
- init: create a new roberta-base model for each fold
- setup: using **AdamW optimizer** and **learning rate scheduler**
- train: each epochs, calculate loss and update weights
- validate: evaluate on the validation
- save: only save model weights if it achieves new best f1 score in that fold

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# k-fold parameter
N_FOLDS = 5
skf = StratifiedKFold(n_splits = N_FOLDS, shuffle = True, random_state = 42)
num_labels = len(label2id)
num_epochs = 3

# store the best f1 of each fold
fold_best_f1s = []

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df_bert, train_df_bert["label"])):
    print(f"\nFold {fold+1}/{N_FOLDS}")
    
    # slice the dataframe
    fold_train_df = train_df_bert.iloc[train_idx].reset_index(drop = True)
    fold_val_df = train_df_bert.iloc[val_idx].reset_index(drop = True)
    
    # dataset and DataLoader
    train_dataset = EmotionDataset(fold_train_df)
    val_dataset = EmotionDataset(fold_val_df)
    train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True, collate_fn = collate_fn)
    val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, collate_fn = collate_fn)
    
    # initialize model for each fold
    model = AutoModelForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels = num_labels,
        id2label = id2label,
        label2id = label2id
    ).to(device)
    
    # optimizer and learning rate scheduler
    optimizer = AdamW(model.parameters(), lr = 2e-5, weight_decay = 0.01)
    total_steps = len(train_loader) * num_epochs
    warmup_steps = int(0.1 * total_steps)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps = warmup_steps, num_training_steps = total_steps
    )
    
    # training loop
    best_val_f1 = -1.0
    best_checkpoint_path = f"model_fold_{fold}.pt"
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0
        
        for batch in tqdm(train_loader, desc = f"fold {fold+1} epoch {epoch+1}"):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            # forward pass
            outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
            loss = outputs.loss
            loss.backward() # backward pass

            # gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
            optimizer.step()
            scheduler.step()
            total_train_loss += loss.item()
            
        avg_train_loss = total_train_loss / len(train_loader)
        
        # validation
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)
                
                outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
                loss = outputs.loss

                # get predictions
                preds = torch.argmax(outputs.logits, dim = -1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        # macro F1 Score
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        val_f1 = f1_score(all_labels, all_preds, average = "macro")
        
        print(f"epoch {epoch+1}: train loss = {avg_train_loss:.4f}, val f1 = {val_f1:.4f}")

        # save best model if need
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), best_checkpoint_path)
            print(f"Saved best model for fold {fold+1}")
            
    fold_best_f1s.append(best_val_f1)
    
    del model, optimizer, scheduler, train_loader, val_loader
    torch.cuda.empty_cache()

print("average f1 score:", np.mean(fold_best_f1s))


Fold 1/5


2025-12-01 12:26:10.900408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764591971.268672      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764591971.389990      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

epoch 1: train loss = 1.0142, val f1 = 0.5068
Saved best model for fold 5


fold 5 epoch 2: 100%|██████████| 2395/2395 [05:56<00:00,  6.73it/s]


epoch 2: train loss = 0.7949, val f1 = 0.5090
Saved best model for fold 5


fold 5 epoch 3: 100%|██████████| 2395/2395 [05:55<00:00,  6.73it/s]


epoch 3: train loss = 0.6726, val f1 = 0.5189
Saved best model for fold 5
average f1 score: 0.5329422213969982


### Create test dataset and DataLoader
- Define a TestDataset class for test set, which does not require labels.
- Initialize the test dataset and its DataLoader.

In [12]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.texts = df["clean_text"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx]}

# create test dataset and DataLoader
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False, collate_fn = collate_fn)

print("test dataset size:", len(test_dataset))

test dataset size: 16281


### Predict using ensemble approach

- predict probabilities for test set with each model
- then average the probabilities of N models
- select the one with the highest average to get more robust probability

In [ ]:
import torch.nn.functional as F

all_fold_probs = []

for fold in range(N_FOLDS):
    print(f"predict with fold {fold+1} model:")
    checkpoint_path = f"model_fold_{fold}.pt"
    
    # initialize model
    model = AutoModelForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels = len(label2id),
        id2label = id2label,
        label2id = label2id
    ).to(device)
    
    # the best weights for this fold
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()
    
    fold_probs = []
    
    # prediction
    with torch.no_grad():
        for batch in tqdm(test_loader, desc = f"fold {fold+1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            # use softmax to get probability distribution
            probs = F.softmax(outputs.logits, dim = -1)
            fold_probs.append(probs.cpu().numpy())
            
    # concatenate all batch predictions
    all_fold_probs.append(np.concatenate(fold_probs))
    
    del model
    torch.cuda.empty_cache()

# ensemble, take the average of probability and select the maximum
avg_probs = np.mean(all_fold_probs, axis = 0)
final_preds = np.argmax(avg_probs, axis = -1)

# id back to labels
pred_labels = [id2label[p] for p in final_preds]
id_to_pred = {id_: label for id_, label in zip(test_df["id"], pred_labels)}

# submission file
sample_sub = pd.read_csv(f"{path}/samplesubmission.csv")
submission = sample_sub[["id"]].copy()
submission["emotion"] = submission["id"].map(id_to_pred)
submission.to_csv("submission.csv", index = False)

predict with fold 0 model:


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
fold 1: 100%|██████████| 1018/1018 [00:43<00:00, 23.42it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


predict with fold 1 model:


fold 2: 100%|██████████| 1018/1018 [00:43<00:00, 23.39it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


predict with fold 2 model:


fold 3: 100%|██████████| 1018/1018 [00:43<00:00, 23.41it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


predict with fold 3 model:


fold 4: 100%|██████████| 1018/1018 [00:43<00:00, 23.40it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


predict with fold 4 model:


fold 5: 100%|██████████| 1018/1018 [00:43<00:00, 23.43it/s]
